In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import pandas as pd

import config
from dxlib.simulation import SimulationManager, Strategy
from dxlib import Portfolio, Signal, TradeType
from dxlib.api import AlpacaMarketsAPI

import pprint
pp = pprint.PrettyPrinter(width=100)

In [14]:
api = AlpacaMarketsAPI(config.alpaca_market_api_key, config.alpaca_market_api_secret)
historical_bars = api.get_historical_bars("AAPL", "2021-03-01")

In [15]:
class HeuristicTest(Strategy):
    def __init__(self):
        super().__init__()
        self.signal_history = []
        self.signal_buy_history = []

    def fit(self):
        pass

    def predict(self):
        pass

    def execute(self, row, idx, history) -> list[Signal]:
        row_signal = pd.Series(index=range(len(row)))
        if (row['AAPL'] > 190) and (len(self.signal_buy_history) < 100):
            row_signal[0] = Signal(TradeType.BUY, 1, row['AAPL'])
            self.signal_buy_history.append(row_signal[0])
        row_signal[pd.isna(row_signal)] = Signal(TradeType.WAIT)
        self.signal_history.append(row_signal)
        return row_signal

In [16]:
starting_cash = 1e6
portfolio = Portfolio()
portfolio.add_cash(starting_cash)

strategy = HeuristicTest()
simulation = SimulationManager(portfolio, strategy, historical_bars["Close"])

In [17]:
historical_quantity = simulation.execute()
print(historical_quantity)
portfolio.print_transaction_history()
print("Profit: ", starting_cash - portfolio.current_value)

Ticker  AAPL
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
..       ...
610     20.0
611     21.0
612     21.0
613     21.0
614     21.0

[615 rows x 1 columns]
0 BUY: cash 1000000.0 @ 1
1 SELL: cash 193.89 @ 1
2 BUY: AAPL 1 @ 193.88
3 SELL: cash 192.47 @ 1
4 BUY: AAPL 1 @ 192.46
5 SELL: cash 191.34 @ 1
6 BUY: AAPL 1 @ 191.33
7 SELL: cash 191.82 @ 1
8 BUY: AAPL 1 @ 191.81
9 SELL: cash 190.69 @ 1
10 BUY: AAPL 1 @ 190.68
11 SELL: cash 190.54999999999998 @ 1
12 BUY: AAPL 1 @ 190.54
13 SELL: cash 190.7 @ 1
14 BUY: AAPL 1 @ 190.69
15 SELL: cash 194.0 @ 1
16 BUY: AAPL 1 @ 193.99
17 SELL: cash 193.73999999999998 @ 1
18 BUY: AAPL 1 @ 193.73
19 SELL: cash 195.10999999999999 @ 1
20 BUY: AAPL 1 @ 195.1
21 SELL: cash 193.14 @ 1
22 BUY: AAPL 1 @ 193.13
23 SELL: cash 191.95 @ 1
24 BUY: AAPL 1 @ 191.94
25 SELL: cash 192.76 @ 1
26 BUY: AAPL 1 @ 192.75
27 SELL: cash 193.63 @ 1
28 BUY: AAPL 1 @ 193.62
29 SELL: cash 194.51 @ 1
30 BUY: AAPL 1 @ 194.5
31 SELL: cash 193.23 @ 1
32 BUY: AAP